In [4]:
import os

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\Krish Goyani\\Desktop\\DS Projects\\CardVista-A-View-into-Unsupervised-Credit-Card-Segmentation'

In [7]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [8]:
from src.CardVista.constants import *
from src.CardVista.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.CardVista.logging import logger
from src.CardVista.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_config = DataIngestion(config=data_ingestion_config)
    data_ingestion_config.download_file()
    data_ingestion_config.extract_zip_file()

except Exception as e :
    raise e

[2024-02-16 16:48:51,330 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-16 16:48:51,339 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-02-16 16:48:51,347 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-16 16:48:51,350 : INFO : common : created directory at: artifacts]
[2024-02-16 16:48:51,350 : INFO : common : created directory at: artifacts/data_ingestion]
[2024-02-16 16:48:53,460 : INFO : 3552199767 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 348486
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d923a2d7c2f227cb9792205312b8de1a7f2676a30be4636beab234568dbb31fd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E2E2:1F970:2698F2:37AB61:65CF449F
Accept-Ranges: b